# Language model in character level 

## Preprocessing of data

In [1]:
import numpy as np
import pandas as pd
import nltk, itertools, re

# data = open('data/reddit-comments-2015-08.csv', 'r').read()

df = pd.read_csv('data/reddit-comments-2015-08.csv')

# with open('data/reddit-comments-2015-08.csv', 'r') as f:
#     reader = 


In [2]:

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
#     string = re.sub(r"\\", " ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip()

df.body = [clean_str(s.strip()) for s in df.body]


In [3]:
#if read from a plain text file
data = open('data/text.txt', 'r').read()
data


'I joined a new league this year and they have\n'

In [4]:
data = '. '.join(df.body)
chars = list(set(data))
print(chars)
print("Data has %d characters in which %d are unique" %(len(data), len(chars)))

['m', '?', 'l', 'j', '4', '`', 'S', 'y', '.', 'L', '8', 'E', 'r', 'H', 'k', ' ', 'p', 't', "'", 'O', '\\', 'v', 'u', 'q', 'D', 'A', 'J', 'Y', 's', 'P', 'd', '6', ')', 'M', 'z', '7', 'i', 'w', 'X', 'N', 'C', 'g', 'c', '5', 'Q', 'F', 'b', '!', 'B', '(', 'K', 'h', '2', '3', '9', 'T', 'o', 'f', ',', '0', 'R', 'U', 'I', 'e', 'G', 'x', 'Z', 'W', 'n', 'a', 'V', '1']
Data has 7515268 characters in which 72 are unique


In [5]:
# create char-to-index, index-to-char lists
char_to_index = {c:i for i, c in enumerate(chars)}
index_to_char = {i:c for i, c in enumerate(chars)}
print(char_to_index, "\n", index_to_char)

{'m': 0, '?': 1, 'l': 2, 'j': 3, '4': 4, '`': 5, 'S': 6, 'y': 7, '.': 8, 'L': 9, '8': 10, 'E': 11, 'r': 12, 'H': 13, 'k': 14, ' ': 15, 'p': 16, 't': 17, "'": 18, 'O': 19, '\\': 20, 'v': 21, 'u': 22, 'q': 23, 'D': 24, 'A': 25, 'J': 26, 'Y': 27, 's': 28, 'P': 29, 'd': 30, '6': 31, ')': 32, 'M': 33, 'z': 34, '7': 35, 'i': 36, 'w': 37, 'X': 38, 'N': 39, 'C': 40, 'g': 41, 'c': 42, '5': 43, 'Q': 44, 'F': 45, 'b': 46, '!': 47, 'B': 48, '(': 49, 'K': 50, 'h': 51, '2': 52, '3': 53, '9': 54, 'T': 55, 'o': 56, 'f': 57, ',': 58, '0': 59, 'R': 60, 'U': 61, 'I': 62, 'e': 63, 'G': 64, 'x': 65, 'Z': 66, 'W': 67, 'n': 68, 'a': 69, 'V': 70, '1': 71} 
 {0: 'm', 1: '?', 2: 'l', 3: 'j', 4: '4', 5: '`', 6: 'S', 7: 'y', 8: '.', 9: 'L', 10: '8', 11: 'E', 12: 'r', 13: 'H', 14: 'k', 15: ' ', 16: 'p', 17: 't', 18: "'", 19: 'O', 20: '\\', 21: 'v', 22: 'u', 23: 'q', 24: 'D', 25: 'A', 26: 'J', 27: 'Y', 28: 's', 29: 'P', 30: 'd', 31: '6', 32: ')', 33: 'M', 34: 'z', 35: '7', 36: 'i', 37: 'w', 38: 'X', 39: 'N', 40: 'C

## Model

In [6]:
hidden_size = 200 # nb of neurons in hidden layer
seq_length = 25 # number of steps to unroll the RNN for, and this is also nb of chars putting in the input of RNN
learning_rate = 1e-1

data_size = len(data)
vocab_size = len(chars) 

#init parameters
W_hh = np.random.randn(hidden_size, hidden_size) #(H,H)
W_xh = np.random.randn(vocab_size, hidden_size) #(D,H)
W_hy = np.random.randn(hidden_size, vocab_size) #(H,M), M=D
b_h = np.zeros((1,hidden_size)) #(H,)
b_y = np.zeros((1,vocab_size)) #(M,)


## Layers

In [7]:
#Reference from @Karpathy: https://gist.github.com/karpathy/d4dee566867f8291f086
#
#                         [b_h]                                                [b_y]
#    w2v                    v                                (h_next)            v
#  x --> x_s -> [W_xh] -> [sum] -> h_raw -> [nonlinearity] -> h_s -> [W_hy] -> [sum] -> y_s -> [exp(y[k])/sum(exp(y))] -> p_s
#                           ^                                  |
#                           '----h_prev------[W_hh]------------'
#


### Forward

In [91]:
def rnn_forward_single_step(x, h_prev, W_xh, W_hh, W_hy, b_h, b_y):
    """
    Run the forward pass for a single timestep of a vanilla RNN that uses a tanh
    activation function.

    The input data has dimension D, the hidden state has dimension H, and we use
    a minibatch size of N.

    Inputs:
    - x: Input data for this timestep, of shape (N, D).
    - h_prev: Hidden state from previous timestep, of shape (N, H)
    - W_xh: Weight matrix for input-to-hidden connections, of shape (D, H)
    - W_hh: Weight matrix for hidden-to-hidden connections, of shape (H, H)
    - W_hy: Weight matrix for hidden-to-output connections, of shape (H, M)
    - b_h: Biases of shape (H,)
    - b_y: Bias of shape (M, )

    Returns a tuple of:
    - h_next: Next hidden state, of shape (N, H)
    - y_s: output of this timestep (N, M)
    - cache: Tuple of values needed for the backward pass.
    """
    
    h_raw = np.dot(x, W_xh) + np.dot(h_prev, W_hh) + b_h  #(N,D)x(D,H) + (N,H)x(H,H) +(1,H) = (N,H)
    h_next = np.tanh(h_raw) #(N, H)
    y_s = np.dot(h_next, W_hy) + b_y #(N, H)x(H,M) +(1,M) =(N,M)
#     p_s = np.exp(y_s) / np.sum(np.exp(y_s), axis=0)
    
    cache = (x, h_prev, h_next, W_xh, W_hh, W_hy, b_h, b_y)
    return h_next, y_s, cache


def rnn_forward(x, h0, W_xh, W_hh, W_hy, b_h, b_y):
    """
    Run a vanilla RNN forward on an entire sequence of data. We assume an input
    sequence composed of T vectors (each vector represents a word/char), each of dimension D. 
    The RNN uses a hidden
    size of H, and we work over a minibatch containing N sequences. After running
    the RNN forward, we return the hidden states for all timesteps.
    Inputs:
    - x: Input data for the entire timeseries, of shape (N, T, D).
    - h0: Initial hidden state, of shape (N, H)
    - W_xh: Weight matrix for input-to-hidden connections, of shape (D, H)
    - W_hh: Weight matrix for hidden-to-hidden connections, of shape (H, H)
    - W_hy: Weight matrix for hidden-to-output connections, of shape (H, M)
    - b_h: Biases of shape (H,)
    - b_y: Bias of shape (M, )
    Returns a tuple of:
    - h: Hidden states for the entire timeseries, of shape (N, T, H)
    - y: Output of the entire timeseries, of shape (N, T, M)
    - cache: Values needed in the backward pass
    """
    N, T, D = x.shape
    H, M = W_hy.shape
    
    h = np.empty((N, T, H))
    cache = {}
    y_s = np.empty((N, T, M))
    
    for i in range(T):
        if i==0: 
            h[:, i, :], y_s[:, i, :], cache[i] = rnn_forward_single_step(x, h0, W_xh, W_hh, W_hy, b_h, b_y)
        else: 
            h[:, i, :], y_s[:, i, :], cache[i] = rnn_forward_single_step(x, h[:, i-1, :], W_xh, W_hh, W_hy, b_h, b_y)
    
    return h, y_s, cache




### Backward

In [8]:
#Reference from @Karpathy: https://gist.github.com/karpathy/d4dee566867f8291f086
#
#                         [b_h] (1,H)                                         [b_y] (1,M)
#    w2v                    v      (N,H)                     (N,H)   (H,M)       v
#  x --> x_s -> [W_xh] -> [sum] -> h_raw -> [nonlinearity] -> h_s -> [W_hy] -> [sum] -> y_s -> [exp(y[k])/sum(exp(y))] -> p_s
# (N,D)         (D,H)       ^                                  |                       (N,M)
#                           '----h_prev------[W_hh]------------'
#                                (N,H)       (H,H)

def rnn_backward_single_step (dh_next, dy, cache):
    """
    Backward pass for a single timestep of a vanilla RNN.
    Inputs:
    - dh_next: Gradient of loss with respect to next hidden state (N, H)
    - dy: of shape (N,M)
    - cache: Cache object from the forward pass
    Returns a tuple of:
    - dx: Gradients of input data, of shape (N, D)
    - dh_prev: Gradients of previous hidden state, of shape (N, H)
    - dWxh: Gradients of input-to-hidden weights, of shape (D, H)
    - dWhh: Gradients of hidden-to-hidden weights, of shape (H, H)
    - dWhy: Gradients of hidden-to-output weights, of shape (H, M)
    - dbh: Gradients of bias vector, of shape (H,)
    - dby: Gradients of bias vector, of shape (M,)
    
    """
    dx, dh_prev, dWxh, dWhh, dWhy, dbh, dby = None, None, None, None, None, None, None
    x, h_prev, h_next, W_xh, W_hh, W_hy, b_h, b_y = cache
    
    
    
    dby = np.sum(dy, axis=0) #(1,M)
    dWhy = np.dot(h_next.T, dy) #(H,N)x(N,M) = (H,M)
    dh = np.dot(dy, Why.T) + dh_next # backprop into h, (N,M)x(M,H)=(N,H)
    dh_raw = (1 - h_next ** 2) * dh # note: tanh(x)' = 1 - tanh^2(x), shape=(N,H)
    dbh = np.sum(dh_raw, axis=0) #(1,H)
    dWxh = np.dot(x.T, dh_raw) # (D,N)x(N,H) = (D,H)
    dWhh = np.dot(h_prev.T, dh_raw) # (H,N)x(N,H)=(H,H)
    dx = np.dot(dh_raw, W_xh.T) # (N,H)x(H,D) = (N,D)
    dh_prev = np.dot(dh_raw, W_hh.T) # (N,H)x(H,H) = (N,H)
    
    ########
#     dpre_actv = (1 - next_h ** 2) * dnext_h         # (N, H)
#     dx = dpre_actv.dot(Wx.T)
#     dprev_h = dpre_actv.dot(Wh.T) #(N,H)x(H,H)=(N,H)
#     dWx = x.T.dot(dpre_actv)
#     dWh = prev_h.T.dot(dpre_actv)
#     db = np.sum(dpre_actv, 0)
    
    
#     dWhy += np.dot(dy, hs[t].T)
#     dby += dy
#     dh = np.dot(Why.T, dy) + dhnext # backprop into h
#     dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
#     dbh += dhraw
#     dWxh += np.dot(dhraw, xs[t].T)
#     dWhh += np.dot(dhraw, hs[t-1].T)
#     dhnext = np.dot(Whh.T, dhraw)
    
    return dx, dh_prev, dWxh, dWhh, dWhy, dbh, dby


def rnn_backward(dh, dy, cache):
    """
    Compute the backward pass for a vanilla RNN over an entire sequence of data.
    Inputs:
    - dh: Upstream gradients of all hidden states, of shape (N, T, H)
    - dy: Upstream gradients of output, of shape (N, M)
    - cache
    Returns a tuple of:
    - dx: Gradient of inputs, of shape (N, T, D)
    - dh0: Gradient of initial hidden state, of shape (N, H)
    - dWx: Gradient of input-to-hidden weights, of shape (D, H)
    - dWh: Gradient of hidden-to-hidden weights, of shape (H, H)
    - dbh: Gradient of biases, of shape (H,)
    - dby: Gradient of biases, of shape (M,)
    """
    N, T, H =dh.shape
    N, M = dy.shape
    
    
    dx = np.empty((N,T,D))
    dWxh = np.zeros((D,H))
    dWhh = np.zeros((H,H))
    dWhy = np.zeros((H,M))
    dbh = np.zeros((1,H))
    dby = np.zeros((1,M))
    dh_prev = 0
    
    for i in reversed(range(T)):
        # Add the current timestep upstream gradient to previous calculated dh
        dh_next = dh_prev + dh[:,i,:]
        dx[:,i,:], dh_prev, dWxh_temp, dWhh_temp, dWhy_temp, dbh_temp, dby_temp = \
                rnn_backward_single_step(dh_next, dy[:,i,:], cache[i])
        dWxh += dWxh_temp
        dWhy += dWhy_temp
        dWhh += dWhh_temp
        dbh += dbh_temp
        dby += dby_temp
    
    dh0 = dh_prev
    return dx, dh0, dWxh, dWhh, dWhy, dbh, dby
    

## Loss

In [ ]:
def temporal_softmax_loss (y_s, y, mask, verbose=False):
    """
    A temporal version of softmax loss for use in RNNs. We assume that we are
    making predictions over a vocabulary of size M for each timestep of a
    timeseries of length T, over a minibatch of size N. The input y_s gives SCORES
    for all vocabulary elements at all timesteps, and y gives the INDICES of the
    ground-truth element at each timestep. We use a cross-entropy loss at each
    timestep, summing the loss over all timesteps and averaging across the minibatch.
    As an additional complication, we may want to ignore the model output at some
    timesteps, since sequences of different length may have been combined into a
    minibatch and padded with NULL tokens. The optional mask argument tells us
    which elements should contribute to the loss.
    Inputs:
    - y_s: Input scores, of shape (N, T, M)
    - y: Ground-truth indices, of shape (N, T) where each element is in the range
         0 <= y[i, t] < M
    - mask: Boolean array of shape (N, T) where mask[i, t] tells whether or not
      the scores at y_s[i, t] should contribute to the loss.
    Returns a tuple of:
    - loss: Scalar giving loss
    - dy_s: Gradient of loss with respect to scores y_s.
    """
    
    N, T, M = y_s.shape

    ys_flat = y_s.reshape(N * T, M)
    y_flat = y.reshape(N * T)
    mask_flat = mask.reshape(N * T)

    probs = np.exp(ys_flat - np.max(ys_flat, axis=1, keepdims=True))
    probs /= np.sum(probs, axis=1, keepdims=True)
    loss = -np.sum(mask_flat * np.log(probs[np.arange(N * T), y_flat])) / N
    dys_flat = probs.copy()
    dys_flat[np.arange(N * T), y_flat] -= 1
    dys_flat /= N
    dys_flat *= mask_flat[:, None]

    if verbose: print('dys_flat: ', dys_flat.shape)

    dy_s = dys_flat.reshape(N, T, M)

    return loss, dy_s
    

    
def loss(X, y=None):
    """
    Compute training-time loss.
    Inputs:
    - X: Input (e.g., image features), of shape (N, D)
    - y: Ground-truth texts (e.g., captions); an integer array of shape (N, T) where
        each element is in the range 0 <= y[i, t] < M
    Returns:
        If y is None, then run a test-time forward pass of the model and return:
        - scores: Array of shape (N, M) giving classification scores, where
          scores[i, c] is the classification score for X[i] and the word/char with index c.

        If y is not None, then run a training-time forward and backward pass and
        return a tuple of:
        - loss: Scalar value giving the loss
        - grads: Dictionary of gradients 
    """
    
    # Cut y into two pieces: text_in has everything but the last word/char
    # and will be input to the RNN; text_out has everything but the first
    # word/char and this is what we will expect the RNN to generate. These are offset
    # by one relative to each other because the RNN should produce word/char (t+1)
    # after receiving word/char (t). The first element of text_in will be the START
    # token, and the first element of text_out will be the first word/char, 
    # and the last element of text_out is END_TOKEN
        
    text_in = y[:, :-1]
    text_out = y[:, 1:]
    
    mask = (text_out != self._null)
    
    
    scores = rnn_forward()

## Build model (classifier)

In [ ]:
class LanguageModel